In [1]:
import os,sys
sys.path.append('../src')

import MSSP


In [2]:
gdir = '/Users/brandon/Documents/GitHub/SNAP Fisheries Spreadsheets/json/current'
outdir = '/Users/brandon/Documents/GitHub/SNAP Fisheries Spreadsheets/json/2016-07-07'

In [3]:
Eq = MSSP.MsspFromJson(gdir)

In [4]:
def t_gen():
    for i, k in enumerate(Eq._targets):
        if isinstance(k, MSSP.mssp_objects.MsspTarget):
            yield i,k
def q_gen():
    for i, k in enumerate(Eq._questions):
        if isinstance(k, MSSP.mssp_objects.MsspQuestion):
            yield i,k

In [5]:
t_cats = [k.category for i,k in t_gen()]
q_cats = [k.category for i,k in q_gen()]

In [6]:
len(t_cats)

134

In [7]:
len(set(t_cats))

24

In [8]:
len(q_cats)

142

In [9]:
len(set(q_cats))

25

In [11]:
set(q_cats).intersection(set(t_cats))

{None}

In [12]:
Eq.attributes(list(set(q_cats)))

AttributeID 40e80940-deaf-3540-8d36-149fa7b86ec5: Strength of governance
AttributeID 739eb9ed-2ac0-3bd1-9f55-2848a1d5cb1a: Spatial coverage/cooperation
AttributeID b9d2bfc0-695e-38a2-9c4c-c40602cfdcf8: STAGNANT ATTRIBUTE WHOSE RESPONSES SHOULD POP UP AS WARNINGS
AttributeID 9ca754e9-4828-36eb-bc82-e9ddfe2d8436: Expert Judgement
AttributeID 83cccd99-f09f-3fa8-9717-d5aa8de211fd: ="Rows "&ROW(F56)&" or "&ROW(F57)&" or "&ROW(F58)&" or "&ROW(F60)&" or "&ROW(F61)
AttributeID 91c55587-a6f4-3d41-9b89-a8b153bee2e7: Enforcement capability
AttributeID ee40416c-8196-30a3-9555-fd9099e6490d: Indices
AttributeID 80bd0ac5-dbc3-3f91-8c24-e813684e06a0: Operational characteristics
AttributeID 522922da-1f18-3a61-b017-8e19e73e3454: Biology
AttributeID 0db86b8d-ead4-3a11-ab5a-8fcf86e14827: ="Rows "&ROW(F54)&" or "&ROW(F55)
AttributeID 12ace8bf-0d5b-3ce6-be67-5e432f992854: Additional requirements (not elsewhere included). These are not included in the vector matching but should pop up as warnings depending o

Those need some work-- all the "&ROW(F60)" and so on (satisfy questions) need to inherit cats from their satisfiers.
the "none"s need work.  everything else is good.

In [10]:
Eq.attributes(list(set(t_cats)))

AttributeID 860e8f82-b106-33d1-9896-c05a8c8edf36: 11 Overrides in case of exceptional circumstances
AttributeID cd5a42d3-9ad8-3a17-ae00-439d2af5c4f3: 8 Sex regulations
AttributeID 83ed4114-c038-3c46-bb46-b9625fb5ce45: Reference points/stock status
AttributeID 3fb33c6e-a363-3eb3-8c21-4b66c73d170f: 7 Size limits (NB may be achieved via gear/spatial/temporal restrictions) - pertaining to controlling selectivity (e.g. protecting juveniles, or oldest (largest) fish that have highest reproductive contribution)
AttributeID b4b4fa69-74b9-3085-af8b-ff388e413b8d: 13 Levies, taxes (e.g. as incentives to avoid areas)
AttributeID 0a64bfe8-b2fd-38e1-b267-9d7400ca5835: 6 Temporal restrictions (can be invoked or modified by HCRs)
AttributeID 6a089a6f-ddbb-360d-a3da-9aca899b1689: 10 Apply additional (precautionary) buffers/adjustments to chosen measure (e.g. catch, effort, size limits, closures)
AttributeID 86eff59e-9877-3edb-9690-7c9184012b43: 4 Other gear controls not related to selectivity (gear TYP

Those all look good, except for the "none"

### Satisfy questions should inherit categories

In [13]:
for i, q in q_gen():
    if len(q.satisfied_by) != 0:
        sat = list(q.satisfied_by)[0]
        print('upgrading category for %d (%s)\n   with %d (%s)' %(i, Eq.attribute(q.category), 
                                                                sat, Eq.attribute(Eq._questions[sat].category)))
        q.category = Eq._questions[sat].category

upgrading category for 30 (="Rows "&ROW(F51)&" or "&ROW(F54)&"-"&ROW(F62))
   with 11 (Indices)
upgrading category for 31 (="Rows "&ROW(F56)&" or "&ROW(F58))
   with 16 (Indices)
upgrading category for 32 (="Rows "&ROW(F54)&" or "&ROW(F55))
   with 14 (Indices)
upgrading category for 33 (="Rows "&ROW(F52)&" or "&ROW(F54)&" or "&ROW(F55))
   with 12 (Indices)
upgrading category for 34 (="Rows "&ROW(F60)&" or "&ROW(F61)&" or "&ROW(F62))
   with 20 (Indices)
upgrading category for 35 (="Rows "&ROW(F56)&" or "&ROW(F57)&" or "&ROW(F58)&" or "&ROW(F60)&" or "&ROW(F61))
   with 16 (Indices)
upgrading category for 36 (="Rows "&ROW(F40)&"-"&ROW(F43))
   with 0 (Biology)
upgrading category for 97 (None)
   with 98 (Data availability)
upgrading category for 162 (Operational characteristics)
   with 112 (Operational)


Now we want to undo the change for 162...

In [14]:
Eq.attribute(Eq._questions[164].category)

u'Operational characteristics'

In [17]:
Eq._questions[162].category = Eq._questions[164].category

### Supply categories for None-category items

In [19]:
tn = [i for i,t in t_gen() if t.category is None]

In [20]:
tn

[48]

In [21]:
Eq.attribute(Eq._targets[tn[0]].title)

u'Catch-based method for classifying stock status (Hilborn 2014 - Nature)'

don't know how to categorize that, since it was added later

In [22]:
qn = [(i,q) for i,q in q_gen() if q.category is None]

In [23]:
[(i, q.references) for i,q in qn]

[(38, [(u'Assessment', (78, None))]),
 (39, [(u'Assessment', (79, None))]),
 (40, [(u'Assessment', (80, None))]),
 (41, [(u'Assessment', (81, None))]),
 (42, [(u'Assessment', (82, None))]),
 (43, [(u'Assessment', (83, None))]),
 (44, [(u'Assessment', (84, None))]),
 (45, [(u'Assessment', (85, None))]),
 (46, [(u'Assessment', (86, None))]),
 (50, [(u'Assessment', (90, None))]),
 (51, [(u'Assessment', (91, None))]),
 (52, [(u'Assessment', (92, None))]),
 (53, [(u'Assessment', (93, None))]),
 (54, [(u'Assessment', (94, None))]),
 (55, [(u'Assessment', (95, None))]),
 (56, [(u'Assessment', (96, None))]),
 (57, [(u'Assessment', (97, None))]),
 (58, [(u'Assessment', (98, None))]),
 (59, [(u'Assessment', (99, None))]),
 (60, [(u'Assessment', (100, None))]),
 (61, [(u'Assessment', (101, None))]),
 (62, [(u'Assessment', (102, None))]),
 (64, [(u'Assessment', (104, None))]),
 (66, [(u'Assessment', (106, None))]),
 (67, [(u'Assessment', (107, None))]),
 (123, [(u'ControlRules', (68, None))]),
 (1

By inspection, many/most of these have the catch-all "Additional requirements (not elsewhere included)..." category, but either missed it because of merge cells or for some other unknown reason.

That category should apply to [rows] 78, 80-86, 90-101, 104, 106, 107

In [27]:
not_elsewhere = Eq.search('not elsewhere included').attributes[0]

In [28]:
Eq.attribute(not_elsewhere)

u'Additional requirements (not elsewhere included). These are not included in the vector matching but should pop up as warnings depending on the response.'

In [30]:
for i in [38] + range(40,47) + range(50,62) + [64, 66, 67]:
    Eq._questions[i].category = not_elsewhere

In [31]:
[(i,q.references) for i,q in q_gen() if q.category is None]

[(39, [(u'Assessment', (79, None))]),
 (62, [(u'Assessment', (102, None))]),
 (123, [(u'ControlRules', (68, None))]),
 (174, [(u'Monitoring', (None, 39))])]

now we deal with these one at a time

In [34]:
Eq.attributes(Eq.attributes_for(39))

AttributeID 71e525de-31af-3ea2-b801-91d42ef29197: Criterion
AttributeID fadbaa68-4e6e-3d82-806e-df0496a56fa5: !!!!!MAKE THIS A MINIMUM DATA REQUIREMENT!! i.e. to include in "vector matching" exercise. 
AttributeID d1eb67c7-a017-3eb4-b00c-4b73efbdcd2e: Selectivity ogive (0= unknown; 1= able to be inferred; 2 = known)
AttributeID 735ef4ac-a66b-3160-b415-feeb288625bb: What is known about the selectivity ogive?
AttributeID 5be56c16-3ec0-3b49-ab9f-3aa1988d9dda: A2


In [37]:
Eq.del_attribute_mapping(39, Eq.search('!!!!!').attributes[0])

Removed 1 reference


In [40]:
Eq.set_category(39, Eq.find_attribute('General criteria')[0])

In [41]:
[(i,q.references) for i,q in q_gen() if q.category is None]

[(62, [(u'Assessment', (102, None))]),
 (123, [(u'ControlRules', (68, None))]),
 (174, [(u'Monitoring', (None, 39))])]

In [42]:
u = Eq.find_or_create_attribute('ULTIMATELY WILL BE A COST ATTRIBUTE FROM THE COST DATABASE')

In [44]:
Eq.set_category(62,u)

attribute map - 1 found


In [48]:
Eq.set_category(123,Eq.find_attribute('Socio-economic')[0])

attribute map - 1 found


In [49]:
Eq.question(174)

Question ID: 174
  * ADDITIONAL CAVEATS
  * CAVEATS - Second, refine the shortlist of monitoring options according to the below range of criteria
 References: Monitoring Col AM
Valid Answers: [None]
 Satisfied By: None
 Satisfies: None
Has Caveats:
      QuestionID  TargetID AnswerValue  \
2573         174        98        None   
2574         174        99        None   
2575         174       100        None   
2576         174       101        None   
2577         174       102        None   
2578         174       103        None   
2579         174       104        None   
2580         174       105        None   
2581         174       106        None   
2582         174       107        None   
2583         174       108        None   
2584         174       109        None   
2585         174       110        None   
2586         174       111        None   
2587         174       112        None   
2588         174       113        None   
2589         174       114        Non

In [50]:
Eq.set_title(174, Eq.find_attribute('ADDITIONAL CAVEATS')[0])

attribute map - 1 found


In [51]:
Eq.set_category(174, Eq.find_or_create_attribute('Additional information'))

### Supply titles for None-titled items

In [52]:
[i for i, q in q_gen() if q.title is None]

[97, 123]

In [53]:
Eq.question(97)

Question ID: 97
(Category: Data availability)
  * Does assessment suggest a certain form of decision rule? (PENDING OUTCOMES OF ASSESSMENT COMPONENT OF MSSP - 2nd layer?)
  * IF YES
 References: ControlRules Row 32
Valid Answers: ['No', 'Yes']
 Satisfied By: [98, 99, 100, 101, 102, 103, 104, 105, 106]
 Satisfies: None
Has Caveats:
     QuestionID  TargetID AnswerValue  \
753          97        50         Yes   
754          97        51         Yes   
755          97        52         Yes   
756          97        53         Yes   
757          97        54         Yes   
758          97        55         Yes   
759          97        58         Yes   
760          97        59         Yes   
761          97        60         Yes   
762          97        61         Yes   
763          97        64         Yes   
764          97        88         Yes   

                                                 Note  
753                            YES - if catch-related  
754                  

In [57]:
Eq.set_title(97, Eq.search('Does assessment suggest a certain form of decision rule').attributes[0])

attribute map - 1 found


In [61]:
Eq.set_title(123, Eq.search('familiar and').attributes[0])

attribute map - 1 found


In [69]:
[(i, t.reference()) for i, t in t_gen() if t.title is None]

[(88, (u'ControlRules', (None, 43))),
 (89, (u'ControlRules', (None, 44))),
 (90, (u'ControlRules', (None, 45)))]

In [70]:
Eq.set_title(88, Eq.find_or_create_attribute('Retain status quo'), record='target')

In [71]:
Eq.set_title(89, Eq.find_or_create_attribute('Levies, taxes'), record='target')

In [72]:
Eq.set_title(90, Eq.find_or_create_attribute('Additional information'), record='target')

In [73]:
[(i, t.reference()) for i, t in t_gen() if t.title is None]

[]

In [74]:
MSSP.write_to_json(Eq.serialize(), outdir)

Creating 189 questions...
Creating 143 targets...
Creating 528 criteria...
Creating 2054 caveats...
Creating 712 attributes...
Creating 246 notes...
Creating colormap...
Output written to folder /Users/brandon/Documents/GitHub/SNAP Fisheries Spreadsheets/json/2016-07-07.


True